In [3]:
from scipy import stats
import numpy as np
import os
from os.path import join as opj
from multiprocessing import Pool
import multiprocessing.managers
from functools import partial
import nibabel as nib


In [8]:
# read the significance matrix
X = np.load('/home1/varunk/Autism-Connectome-Analysis-bids-related/significance_matrix.npy', mmap_mode='r')
# stack a column indentifying the roi value
X = np.c_[1:X.shape[0]+1,X]
num_ROIs = X.shape[0]

In [4]:
# For passing on the shared file:
class MyManager(multiprocessing.managers.BaseManager):
    pass
MyManager.register('np_zeros', np.zeros, multiprocessing.managers.ArrayProxy)

In [9]:
# Read mask files:
mask_file = os.path.expandvars('$FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz')
mask_data = nib.load(mask_file)
mask = mask_data.get_data()

# read the file on which you want to write the p-values
brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/func/sub-0050952_task-rest_run-1_bold.nii.gz'
brain_data = nib.load(brain_file)
brain = brain_data.get_data()

# Edit header to store ROI correlation maps in t(4th) dimension

(brain_data.header).set_data_shape([91,109,91,num_ROIs])

In [31]:
def do_fdr(fdrcorrected_brain_4Dtensor,fdrcorrected_brain_4Dtensor_mask,brain_voxel_list):
    mask_data = nib.load(mask_file)
    mask = mask_data.get_data()
    
    # to use the deader of an anatomical file tht has float datatype. mask had uint datatype that made all the p-values zero
    brain_file = '/home1/varunk/data/ABIDE-BIDS-Preprocessed/NYU/sub-0050952/anat/sub-0050952_T1w.nii.gz'
    brain_data = nib.load(brain_file)
    
    
    # Two empty brain to store the fdr corrected values and mask (which voxel is correlated)
    brain = brain_data.get_data()
    
    brain_corrected_mask = np.zeros((brain.shape))


    roi_number = int(brain_voxel_list[0]) - 1
    brain_voxel_list = brain_voxel_list[1:]
    
    # FDR Correction: http://www.statsmodels.org/devel/generated/statsmodels.sandbox.stats.multicomp.fdrcorrection0.html#statsmodels.sandbox.stats.multicomp.fdrcorrection0
    
    alpha = 0.05
    from statsmodels.sandbox.stats.multicomp import fdrcorrection0

    brain_voxel_list_mask, brain_voxel_list  = fdrcorrection0(brain_voxel_list,alpha)
    
    
    
    
    
    x_dim, y_dim, z_dim = mask.shape

    brain_voxel_counter = 0
    for i in range(x_dim):
        for j in range(y_dim):
            for k in range(z_dim):
                if mask[i,j,k] == 1:
                    brain[i,j,k] = brain_voxel_list[brain_voxel_counter]
                    if brain_voxel_list_mask[brain_voxel_counter] == True:
                        brain_corrected_mask[i,j,k] = 1.0
                    brain_voxel_counter = brain_voxel_counter + 1
                    

    
    
    if brain_voxel_counter == len(brain_voxel_list):
        
     
    
        print('Brain shape ',brain.shape)
        fdrcorrected_brain_4Dtensor[:,:,:,roi_number] = brain
        fdrcorrected_brain_4Dtensor_mask[:,:,:,roi_number] = brain_corrected_mask

        print('Stored ROI file the shared arrays! ')
        print("Job Done for ROI-",roi_number)
      
        return 1
    else:
        print("p-values fitting failed for ROI-",roi_number)
        return 0

In [32]:
%%time
# Create pool of 8 workers

pool_inputs = [] #np.arange(number_of_ROIs)

for roi in range(num_ROIs):
    pool_inputs.append(X[roi,:])
    

# print (brain_data.header)


m = MyManager()
m.start()
fdrcorrected_brain_4Dtensor = m.np_zeros((brain_data.header.get_data_shape()))
fdrcorrected_brain_4Dtensor_mask = m.np_zeros((brain_data.header.get_data_shape()))

func = partial(do_fdr, fdrcorrected_brain_4Dtensor,fdrcorrected_brain_4Dtensor_mask)

pool = Pool(8)

data_outputs = pool.map(func, pool_inputs)


Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 54
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Job Done for ROI- 63
Job Done for ROI- 0
Stored ROI file the shared arrays! 
Job Done for ROI- 18
Job Done for ROI- 36
Job Done for ROI- 45
Job Done for ROI- 27
Job Done for ROI- 9
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 37
Stored ROI file the shared arrays! 
Job Done for ROI- 64
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared 

Job Done for ROI- 75
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 102
Stored ROI file the shared arrays! 
Job Done for ROI- 93
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 120
Stored ROI file the shared arrays! 
Brain shape  (91, 109, 91)
Job Done for ROI- 129
Stored ROI file the shared arrays! 
Job Done for ROI- 138
Stored ROI file the shared arrays! 
Job Done for ROI- 111
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 85
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 76
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 130
Stored ROI file the shared arrays! 
Job Done for ROI- 94
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Brain shape  (91, 109, 91)
Job Done for ROI

Job Done for ROI- 177
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 168
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 195
Stored ROI file the shared arrays! 
Job Done for ROI- 213
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 204
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 186
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 160
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 151
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Job Done for ROI- 169
Stored ROI file the shared arrays! 
Job Done for ROI- 178
Brain shape  (91, 109, 91)
Brain shape  (91, 109, 91)
Stored ROI file the shared arrays! 
Stored ROI file the shared arrays! 
Job Done for ROI- 214
Job Done for ROI- 196
Brain shape  (91, 109, 91)
Stored ROI file 

In [33]:
# shared arrays are Array proxy objects so convert to np array to save them as brains
fdrcorrected_brain_4Dtensor = np.asarray(fdrcorrected_brain_4Dtensor)
fdrcorrected_brain_4Dtensor_mask = np.asarray(fdrcorrected_brain_4Dtensor_mask)


# Saving the fdr corrected brains: 
brain_with_header = nib.Nifti1Image(fdrcorrected_brain_4Dtensor, affine=brain_data.affine,header = brain_data.header)
nib.save(brain_with_header,'fdr_corrected_brains_2.nii.gz')

brain_with_header = nib.Nifti1Image(fdrcorrected_brain_4Dtensor_mask, affine=brain_data.affine,header = brain_data.header)
nib.save(brain_with_header,'fdr_corrected_brains_mask_2.nii.gz')

In [34]:
np.max(fdrcorrected_brain_4Dtensor_mask)

1.0

In [35]:
np.where(fdrcorrected_brain_4Dtensor_mask[:,:,:,:] == 1)

(array([29, 32, 41, 41, 42, 42, 43, 43, 44, 52, 52, 53]),
 array([29, 74, 75, 75, 75, 75, 75, 75, 75, 76, 76, 76]),
 array([48, 19, 36, 37, 36, 37, 36, 37, 36, 26, 27, 27]),
 array([ 39, 167,  26,  26,  26,  26,  26,  26,  26, 177, 177, 177]))

In [36]:
fdrcorrected_brain_4Dtensor[29,29,48,39]

0.038100983947515488

In [39]:
test_mask = nib.load('fdr_corrected_brains_mask_2.nii.gz')

In [38]:
mask_brain_test = test_mask.get_data()

In [28]:
np.where(mask == 1)

(array([ 9,  9,  9, ..., 80, 80, 80]),
 array([46, 46, 46, ..., 52, 52, 53]),
 array([34, 35, 36, ..., 30, 31, 30]))

In [30]:
!fslhd fdr_corrected_brains.nii.gz

filename       fdr_corrected_brains.nii.gz

sizeof_hdr     348
data_type      FLOAT32
dim0           4
dim1           91
dim2           109
dim3           91
dim4           274
dim5           1
dim6           1
dim7           1
vox_units      mm
time_units     s
datatype       16
nbyper         4
bitpix         32
pixdim0        0.000000
pixdim1        2.000000
pixdim2        2.000000
pixdim3        2.000000
pixdim4        2.000000
pixdim5        1.000000
pixdim6        1.000000
pixdim7        1.000000
vox_offset     7888
cal_max        0.0000
cal_min        0.0000
scl_slope      1.000000
scl_inter      0.000000
phase_dim      0
freq_dim       0
slice_dim      3
slice_name     Unknown
slice_code     0
slice_start    0
slice_end      90
slice_duration 0.000000
time_offset    0.000000
intent         Unknown
intent_code    0
intent_name    
intent_p1      0.000000
intent_p2      0.000000
intent_p3      0.000000
qform_name     MNI_152
qform_code     4
qto_xyz:1      -2.000000  -0.000000  -